In [ ]:
import json
import torch

In [ ]:
with open("./params.json", mode = "r", encoding = "utf-8") as f:
    data = json.load(f)
    model_path = data["model_path"]
    num_single_sample_timesteps = data["num_single_sample_timesteps"]
    input_window_length = data["input_window_length"]
    label_window_length = num_single_sample_timesteps - input_window_length
    input_features = data["input_features"]
    label_features = data["label_features"]

In [ ]:
model = torch.load(model_path, weights_only = False)
model.eval()